#### BERT

In [12]:
import json
import os
import numpy as np
import pandas as pd
import string
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
document_path = '/Users/giudittadelsarto/Desktop/DIS_Project/DIS_code/truncated_documents.json'

In [3]:
with open(document_path) as json_file:
    documents = json.load(json_file)

In [4]:
train_queries_path = '../dis-project-1-document-retrieval/train.csv'

In [5]:
# Load the training queries
train_queries = pd.read_csv(train_queries_path)

In [6]:
def get_doc_by_id(doc_id):
    return next(doc for doc in documents if doc['docid'] == doc_id)

In [7]:
def get_langauge(lang_id):
    """
    Returns the language name given the language id in order to pass it to the stopwords function
    """
    cases = {
        'en' : 'english',
        'de' : 'german',
        'fr' : 'french',
        'es' : 'spanish',
        'it' : 'italian',
        'ar' : 'arabic',
        'ko' : 'korean'
    }
    return cases.get(lang_id, 'unknown') 

In [8]:
def preprocess_text(text, language_id):
    """
    Preprocess the text by removing stopwords, stemming and lemmatizing the text
    """
    # remove punctuation
    text = "".join([ch for ch in text if ch not in string.punctuation])
    # remove stopwords
    stop_words = set(stopwords.words(get_langauge(language_id)))
    words = nltk.word_tokenize(text)
    words = [word for word in words if word.lower() not in stop_words]
    # stemming
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    # lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return words

In [10]:
# def cosine_similarity(a, b):
#     return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [11]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/anaconda3/envs/dis/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
documents_embeddings = []
for doc in tqdm(documents):
    text = doc['text']
    embeddings = model.encode(text)
    documents_embeddings.append(embeddings)

100%|██████████| 1000/1000 [00:38<00:00, 26.14it/s]


In [15]:
train_queries_embeddings = []
gt_documents = []
for _, query in tqdm(train_queries.iterrows()):
    embeddings = model.encode(query['query'])
    gt_documents.append(query['positive_docs'])
    train_queries_embeddings.append(embeddings)

21875it [08:10, 44.62it/s]


In [16]:
len(gt_documents)

21875

In [18]:
len(train_queries_embeddings[0])

768

In [24]:
documents_embeddings_np = np.array(documents_embeddings)

In [25]:
documents_embeddings_np.shape

(1000, 768)

In [27]:
train_queries_embeddings_reshaped = train_queries_embeddings[0].reshape(1, -1)
similarity_scores = cosine_similarity(train_queries_embeddings_reshaped, documents_embeddings_np)

In [29]:
similarity_scores.shape

(1, 1000)

In [30]:
# find index of the 10 most similar documents
most_similar_docs = np.argsort(similarity_scores[0])[-10:]


In [31]:
most_similar_docs

array([ 30, 978, 993, 833, 668, 503, 900, 457, 807, 583])

In [32]:
# recoveer the docids of the most similar documents
most_similar_docids = [documents[i]['docid'] for i in most_similar_docs]
most_similar_docids

['doc-en-10634',
 'doc-en-12492',
 'doc-en-3250',
 'doc-en-16089',
 'doc-en-8017',
 'doc-en-5989',
 'doc-en-9127',
 'doc-en-10148',
 'doc-en-9116',
 'doc-en-161']

In [33]:
# save documents embeddings
np.save('documents_embeddings.npy', documents_embeddings_np)

In [34]:
doc2= np.load('documents_embeddings.npy')

In [36]:
doc2.shape

(1000, 768)

In [ ]:
def create_submission(queries_embeddings, documents_embeddings):
    """
    Create a submission file with the 10 most similar documents for each query
    """
    submission = []
    for i, query_embeddings in enumerate(queries_embeddings):
        similarity_scores = cosine_similarity(query_embeddings.reshape(1, -1), documents_embeddings)
        most_similar_docs = np.argsort(similarity_scores[0])[-10:]
        most_similar_docids = [documents[i]['docid'] for i in most_similar_docs]
        submission.append({
            'query_id': i,
            'retrieved_docs': most_similar_docids
        })
    
    # save the submission file in csv
    submission_df = pd.DataFrame(submission)
    submission_df.to_csv('submission.csv', index=False)
    print('Submission file created successfully')
